# Combining Datasets with Pandas

#### Creation of a new environment which contains geopandas

Why's that?  
In this notebook we want to use the GeoPandas package, which is based on an _open source project to add support for geographic data to pandas objects._ - in brief: we will have a dataframe with an additional geometric datatype.  

Since we usually don't need packages for geospatial data, we don't want to load it everytime we activate our usual nf_base environment.  

Let's create a new environment called 'nf_geo', that has 'geopandas' installed.

In [ ]:
# # clone the sabbys_base - this may take a few seconds up to two minutes ...
# !conda create --yes --name sabby_geo python=3.9 \
#     matplotlib pandas=1.5.2 geopandas scipy seaborn statsmodels scikit-learn ipykernel numpy pip

*(If this last step takes longer than up to a minute and there's a message telling you conda is "solving environment", please reach out to us.)*

in order to use our new environment, you have to switch the kernel and select the python kernel of your new nf_geo environment. if it doesn't appear immediately in the list of available kernels, klick on 'select another kernel'...

---

Now that we have our new nf_geo environment, activate it for this jupyter notebook (choose the kernel) and we're ready to import our needed modules:

In [2]:
# standard import of pandas
import pandas as pd

# additional import of the geopandas package
import geopandas as gpd

# numpy, "numerical python" - we'll cover this in the following notebooks.
import numpy as np

# hides warning messages
import warnings
warnings.filterwarnings("ignore")

## Loading the first dataset
The data we'll use is data on bicycle theft crimes at the granular level of Berlin city planning areas, so-called "LOR" - "Lebensweltlich orientierte Räume", we will stumble over it again later!  
This data is provided by Berlin Open Data and collected by the police of Berlin.  

### The goal of our analysis is: 
### Identify areas in Berlin with the most bike thefts and the bike type that has the highest share in thefts.

But first things first: We make the data accessible just by loading the .csv-file into a dataframe and get an overview.

[Website to datatset -  daten.berlin.de](https://daten.berlin.de/datensaetze/fahrraddiebstahl-berlin)

- Licence:
    - Creative Commons Namensnennung CC-BY License
- Geographical Granularity: 
    - Berlin
- Publisher: 
    - Polizei Berlin LKA St 14
- E Mail: 
    - onlineredaktion@polizei.berlin.de

Next comes a code cell where we define all functions, that we will use in this notebook. 

In [3]:
# Function Definitions

def clean_bike_data(df):
    # columns name to lower case
    df.columns = df.columns.str.lower()
    # drop duplicates
    df = df.drop_duplicates().copy()
    # drop column 'angelegt_am' and 'erfassungsgrund' - irrelevant to us, when and why observation got added to the database.
    df.drop(['angelegt_am', 'erfassungsgrund'], axis=1, inplace=True)
    #df = df.drop(columns=['angelegt_am', 'erfassungsgrund']) # alternative zu zeile drüber
    # we have just 167 attempts and 7 thefts of unknown state in our dataset, so we decide to drop those observations.
    df = df[df['versuch'] != 'Ja']
    df = df[df['versuch'] != 'Unbekannt']
    df.reset_index(inplace=True)
    df.drop('index', axis=1, inplace=True)
    # df.set_index('versuch', inplace=True)
    # df.drop(['Ja', 'Unbekannt'], inplace=True)
    # change date text string to datetime datatype
    df['tatzeit_anfang_datum'] = pd.to_datetime(df['tatzeit_anfang_datum'], format='%d.%m.%Y')
    df['tatzeit_ende_datum'] = pd.to_datetime(df['tatzeit_ende_datum'], format='%d.%m.%Y')
    return df

In [ ]:
thefts_df_raw = pd.read_csv('../../Data/Fahrraddiebstahl.csv', encoding='latin-1') # proper encoding is necessary here!
# make column names lowercase

thefts_df_raw.sample(10)

In [ ]:
# what's the shape, the observations, datatypes and null-counts?
thefts_df_raw.info()

## Clean Data

In [ ]:
# clean the bike data by applying the clean_bike_data function, 
# that you developed in the data_preparation exercise.

# enter code below, assign output to a new variable called thefts_df.
thefts_df = clean_bike_data(thefts_df_raw).copy()
thefts_df

In [ ]:
thefts_df.info()

## Prepare Data

Now that we're done cleaning our dataset, we'll take a look at the unique values of `art_des_fahrrads`...


In [ ]:
# A glance at the values of the type of bikes in the dataframe
thefts_df.art_des_fahrrads.unique()

Great! this is the column we need to specifiy the type of bike that has highest share in our bike thefts! to make our analysis on the bike type even easier we show you a popular data transformation in the next code block...

The idea is to impute it by using categorical data to so called "dummy variables".  
Such a variable (aka indicator variable) is a numeric variable representing categorical data by giving each category an own column and assign a 0 or 1 to it.  

We'll use this on the "Art des Fahrrads" column, the type of bike.

In [ ]:
# get_dummies is a method called on the pandas module - you simply pass in a Pandas Series 
# or DataFrame, and it will convert a categorical variable into dummy/indicator variables. 
# The idea of dummy coding is to convert each category into a new column, and assign a 1 or 0 to the column.
# This can be an important step during data preparation for machine learning.

# creating a dataset of type of bike dummy variables.
biketype_dummies = pd.get_dummies(thefts_df.art_des_fahrrads, prefix='type')
biketype_dummies.sample(10)

This looks good but now `biketype_dummies` is a different dataframe than `thefts_df`, we have our first use case where we need to merge to dataframes.

In [ ]:
print("--------- DF 1 : thefts_df -----------------")
display(thefts_df.head(3))
print("--------- DF 2 : biketype_dummies ----------")
display(biketype_dummies.head(3))

Discuss with your group partner....on what key could you combine the two datasets? ...

## Combining dataframes

In this case we can only join on the index, there is no other unique identifier in the data. As we want to combine via index, which method is preferred `join()` or `merge()`? 

### Join()

Now let's look at the join() method. It joins on indices by default and is called on a dataframe instance. This means that we can simply join our bike type dummies dataframe back to our original bike thefts dataframe with the following code:

In [ ]:
# Joining columns of another DataFrame using the join() method.
join_df = thefts_df.join(biketype_dummies)
join_df.sample(10)

In [ ]:

# Let's have a look at the columns of our newly assigned dataframe
join_df.info()

In [13]:
# let's store our final output in a new variable and create a new copy of the dataset
thefts_df_enriched = join_df.copy()

The arguments of `.join` are the following:
```
DataFrame.join(other, on=None, how='left', lsuffix='', rsuffix='', sort=False)
```
The documentation refers to the second dataframe as 'other', which the documentations of the other combining methods often refer to as 'right'.
With how we can specify which join method we want to use.

If we want to join using a common column, we need to set this column to be the index in both dataframes. The joined DataFrame will have the common column as its index.
```
df.set_index('column_name').join(other.set_index('column_name'))
```
Another option to join using a common column is to use the on parameter. This method preserves the original DataFrame’s index in the result.
```
df.join(other.set_index('column_name'), on='column_name')
```
See the documentation for more information.

The how argument to merge specifies which keys are included in the resulting table. If a key combination does not appear in either the left or right tables, the values in the joined table will be NA. Here is a summary of the how options and their SQL equivalent names:

Merge/Join in Pandas | SQL Join Name | Description
---|---|---
left| LEFT OUTER JOIN | Use keys from left frame only
right | RIGHT OUTER JOIN | Use keys from right frame only
outer | FULL OUTER JOIN | Use union of keys from both frames
inner | INNER JOIN | Use intersection of keys from both frames


You can also think of it as set theory and use Venn diagrams to illustrate what happens in each method.

![Join Methods](../../images/join_types.png)

### Merge()
Let's look at the `merge()` method. Merge combines dataframes on common columns by default and can be used via the pandas module AND called on a dataframe instance.

The arguments of `.merge` are the following: 
````
DataFrame.merge(right, how='inner', on=None, left_on=None, right_on=None, left_index=False, right_index=False, sort=False,   
suffixes=('_x', '_y'), copy=True, indicator=False, validate=None)
````
See the documentation [here](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html).

In [14]:
# Since in both dataframes, we need a common column.
# Let's use the index column as the one to merge on:
# moved this command to the function on top
thefts_df_ind = thefts_df.reset_index()
biketype_dummies_ind = biketype_dummies.reset_index()

In [ ]:
# check result - you will see a new column called index in the dataframe
thefts_df_ind.tail(10)

In [ ]:
# check result - you will see a new column called index in the dataframe
biketype_dummies_ind.tail(10)

In [ ]:
# Merge the quality_dummies df on the thefts_df instance on the common column 'index'
merge_df1 = thefts_df_ind.merge(biketype_dummies_ind, on='index')
merge_df1.sample(10)

In [ ]:
# Or another way: Merge the two dataframes via the pandas module on the common column 'index'
merge_df2 = pd.merge(thefts_df_ind, biketype_dummies_ind, on='index')
merge_df2.tail(10)

The second approach with the `merge()`-method was for showing you the syntax, we will not use the result anywhere. For the rest of the notebook we will work with the combined dataframe from the `join()`-method, remember we called it `thefts_df_enriched`.

## Combining multiple data sources

Remember we initially said, we wanted to be able to identify areas in Berlin with the most bike thefts?  
So far, we can't.  

We have a lot of features describing the actual bike thefts, but we have nothing to really spot the area where it happens. The only thing we have in our dataframe is this suspicious "LOR" - so we have to do some research on it, if and how we can use it ...  

The [dataset description](https://www.berlin.de/polizei/_assets/dienststellen/lka/datensatzbeschreibung.pdf) at Berlin Open Data tells us about the LOR column:
- Kennung des Planungsraums, 8-stellig
- Raumhierarchie lebensweltlich orientierte Räume (LOR) der Senatsverwaltung für Stadtentwicklung und
Wohnen

Wow. _Raumhierarchie lebensweltlich orientierte Räume_ - that's where you know you deal with authorities. 
Since we don't have any Ideas what that means, we google it and find, that at the Website of [stadtentwicklung.berlin.de](https://www.stadtentwicklung.berlin.de/planen/basisdaten_stadtentwicklung/lor/de/download.shtml) there are LOR associated vector data files, .shp "shapefiles". So we have a look at them, too ...


We now access the shapefiles and try to combine them with our biketheft data.

In [ ]:
# assign a geodataframe based on the shapefile
gdf = gpd.GeoDataFrame.from_file('../../Data/LOR_SHP_2021/lor_plr.shp')
gdf.columns = gdf.columns.str.lower()
gdf.head(5)

# attention! if you had troubles installing geopandas, you won't be able to plot the ploygons!
# however, you can still proceed with the rest of the analysis if you read in this csv instead:
# gdf = pd.read_csv('../../data/shapefile.csv')

In [ ]:
gdf.info()

So we see, this gave us a dataframe with obviously the LOR as plr_id, the district name and the geometrical shape of the area as a polygon.

##### Polygon? What was that again?

<img src="../../images/geometries.jpg" alt="geometries" width="500"/>

So, those polygons should give us areas of Berlin. Let's give it a try: 

In [ ]:
# plotting the geometries
berlin = gdf.plot(color='grey', figsize=(12, 12));
# optional: comment out the next line to highlight a particular suburb in red...
gdf[gdf['plr_name'] == 'Waßmannsdorfer Chaussee'].plot(ax=berlin, color='red')


Kepler for beautiful visuals: https://kepler.gl/,
can be used in jupyter notebooks as well.

That somehow looks like Berlin which makes us quite confident to proceed to try to merge the sets, since our bike theft data is not yet inside our geodataframe (or vice versa) - those are still two seperate data sets.  

So - we need to have a look at the column that allow us to merge ...

In [ ]:
# bike thefts lor column
thefts_df.lor.info()

In [ ]:
# geodataframe lor column
gdf.plr_id.info()

Not that easy, again.  
- The column 'lor' in the bike theft data is an integer.  
- Integers as numeric values can't have leading zeros.  
- That's why it sometimes is 8 digits, sometimes is just 7 digits long - it then misses a leading 0 - we need to impute!  

In the geodataframe, the lor column is an object, which means a string in this case.  
Feel free to have a closer look ...

In [ ]:
# merging the two DataFrames by plr_id in format INT
# (so converting in gdf the type of 'plr_id' into int, and just renaming the column 'lor' in thefts)

gdf_int = gdf.copy()
thefts_df_enriched_int = thefts_df_enriched.copy()

gdf_int['plr_id_int'] = gdf_int['plr_id'].astype(int)
thefts_df_enriched_int['plr_id_int'] = thefts_df_enriched_int['lor']

gdf_biketheft_int = pd.merge(thefts_df_enriched_int, gdf_int, on='plr_id_int', how='outer')
gdf_biketheft_int

In [ ]:
# Calculate maximum and minimum length for numeric columns

# The .astype(str) converts the numeric values to strings.
# The .apply(len) calculates the length of each string.

max_lengths = thefts_df_enriched['lor'].astype(str).apply(len).max()
min_lengths = thefts_df_enriched['lor'].astype(str).apply(len).min()

# Print the results
print(f"Maximum length: {max_lengths}")
print(f"Minimum length: {min_lengths}")

In [ ]:
# change the lor column datatype to string and call new column 'lor_str'
thefts_df_enriched['lor_str']= thefts_df_enriched['lor'].astype(str)
thefts_df_enriched

In [ ]:
thefts_df_enriched.lor_str.info()

In [ ]:
# fill leading gaps up to 8 characters with zeros and call the new column accordingly to the geodataframe
# example 3400723 to 03400723

# The zfill() method is used to pad the left side of strings with zeros.
# It ensures that the resulting string has a specified minimum length by adding leading zeros if necessary.
thefts_df_enriched['plr_id'] = thefts_df_enriched['lor_str'].str.zfill(8)
thefts_df_enriched.tail(10)

In [ ]:
#before dropping not needed columns, let's check
thefts_df_enriched[['plr_id','lor_str']].head()

In [ ]:
max_lengths_2 = thefts_df_enriched['plr_id'].astype(str).apply(len).max()
min_lengths_2 = thefts_df_enriched['plr_id'].astype(str).apply(len).min()

# Print the results
print(f"Maximum length: {max_lengths_2}")
print(f"Minimum length: {min_lengths_2}")

In [31]:
#...looks good in our sample check :-)

In [ ]:
# dropping no longer needed columns
thefts_df_enriched.drop(columns=['lor', 'lor_str'], inplace=True)
thefts_df_enriched

In [ ]:
# compare with the geodataframe
display(thefts_df_enriched[['art_des_fahrrads','delikt','plr_id']].head())
display(gdf.head())

Now, we are able to merge our dataframes

In [ ]:
# merge dataframes 'gdf' and 'thefts_df_enriched' on the plr_id columns
# call new dataframe 'gdf_biketheft'
# what type of merge do we want to do? in our case we only want to keep the rows 
# that match with a plr in  gdf, so we perform 'inner' merge

gdf_biketheft = pd.merge(thefts_df_enriched, gdf, on='plr_id', how='outer')
gdf_biketheft

In [ ]:
gdf_biketheft.info()

And so, we are finally able to infer infer the are with the most bikes stolen  
by aggregating count of thefts:

### How many bikethefts per postcode?

In [ ]:
gdf_biketheft.type_Damenfahrrad.value_counts()

In [ ]:
gdf_biketheft.groupby('plr_id').size()

In [ ]:
gdf_biketheft.info()

In [ ]:
# counting thefts in areas
# note: we need .reset_index(name='thefts') to convert the output back to a pandas dataframe.
df_plr_group_thefts = gdf_biketheft.groupby('plr_id')['versuch'].count().reset_index(name='thefts')
# showing new dataframe with plr_id and aggregated count of thefts
df_plr_group_thefts #.tail()

In [ ]:
df_plr_group_thefts[df_plr_group_thefts['thefts']==0]

### What's the average monetary value of the thefts per postcode?

In [ ]:
# count thefts in areas and store the result in a new df called 'df_plr_group_mean',
# name the column with avg monetary value 'avg_amount'
df_plr_group_mean = gdf_biketheft.groupby('plr_id').agg(avg_amount=('schadenshoehe', 'mean'))

# showing new dataframe with plr_id and aggregated mean of thefts
df_plr_group_mean #.head()

### How many thefts per bike type per postcode?

In [42]:
# calculate the number of thefts per bike type per post_code
# and store the result in a new df called 'df_bike_types_count'

#df_bike_types_count = gdf_biketheft.groupby(['plr_id', 'art_des_fahrrads'])['art_des_fahrrads'].count()
#df_bike_types_count

In [ ]:
df_plr_group_thefts.thefts.sum()

In [ ]:
df_bike_types_count_2 = gdf_biketheft.groupby(['plr_id'])\
    .agg({
        'type_Damenfahrrad': 'sum', 
        'type_Fahrrad': 'sum', 
        'type_Herrenfahrrad': 'sum', 
        'type_Kinderfahrrad': 'sum', 
        'type_Lastenfahrrad': 'sum',
        'type_Mountainbike': 'sum',
        'type_Rennrad': 'sum',
        'type_diverse Fahrräder':'sum'
        }).reset_index()
df_bike_types_count_2

In [ ]:
## merging the aggregates into the initial geodataframe

# merge 'df_plr_group_thefts' and 'gdf' and save result as 'gdf_biketheft_1'
gdf_biketheft_1 = pd.merge(df_plr_group_thefts, gdf, on='plr_id')

# merge 'gdf_biketheft_1' and 'df_plr_group_mean' and save result as 'gdf_biketheft_2'
gdf_biketheft_2 = pd.merge(gdf_biketheft_1, df_plr_group_mean, on='plr_id')

# merge 'gdf_biketheft_2' and 'df_bike_types_count' and save result as 'gdf_biketheft'
gdf_biketheft_res = pd.merge(gdf_biketheft_2, df_bike_types_count_2, on='plr_id')
gdf_biketheft_res #.head(5)

In [ ]:
gdf_biketheft_res.thefts.isna().value_counts()

In [ ]:
gdf_biketheft_res[gdf_biketheft_res['thefts']==0]

In [ ]:
gdf_biketheft_res['plr_id'].duplicated().value_counts()

In [ ]:
display(df_plr_group_thefts.head(1))
display(gdf.head(1))
display(df_plr_group_mean.head(1))
display(df_bike_types_count_2.head(1))

In [ ]:
gdf_biketheft_res.info()

### Share of thefts per bike type per postcode

In order to answer one of our main questions we need to calculate the shares for each bike type.


In [51]:
# we need the package numpy to round some numbers
import numpy as np

In [52]:
# give nr of thefts per bike type and total thefts, calculcate the share.
# name the new columns in the following form:
# share_Damenfahrrad = nr type_Damenfahrrad / nr thefts

#gdf_biketheft_res['share_Damenfahrrad'] = (gdf_biketheft_res['type_Damenfahrrad'] / gdf_biketheft_res['thefts']).round(2)
#gdf_biketheft_res.head()

In [ ]:
bike_types = ['Damenfahrrad', 'Fahrrad', 'Herrenfahrrad', 'Kinderfahrrad', 'Lastenfahrrad', 'Mountainbike', 'Rennrad', 'diverse Fahrräder']

for bike_type in bike_types:
    gdf_biketheft_res[f'share_{bike_type}'] = (gdf_biketheft_res[f'type_{bike_type}'] / gdf_biketheft_res['thefts']).round(2)

gdf_biketheft_res.head(12)

In [ ]:
gdf_biketheft_res.type_Herrenfahrrad.sum()

In [ ]:
# let's look at one example to double check our calculations
gdf_biketheft_res[['plr_id','type_Herrenfahrrad','thefts','share_Herrenfahrrad']].sort_values('thefts', ascending=False).head(20)

In [56]:
# Think of more ways to check your calculations!

In [ ]:
# ok great now we need to calculate the bike type with the highest share for each row...
# we want to store this information in a new column called 'type_highest_share'
# tip: search of a pandas function that fullfills this task! 

## with idmax the order of the columns matters: when two columns have the same value, the first one will be picked

gdf_biketheft_res['type_highest_share'] = gdf_biketheft_res[[
    'share_Herrenfahrrad',  'share_Damenfahrrad',
     'share_Kinderfahrrad', 'share_Lastenfahrrad',
     'share_Mountainbike', 'share_Fahrrad',
    'share_Rennrad', 'share_diverse Fahrräder'
    ]].idxmax(axis=1)
gdf_biketheft_res.sample(5)

In [ ]:
##Filter for the following columns

gdf_biketheft_res[[
    'plr_name','share_Rennrad','share_Fahrrad','share_Damenfahrrad','share_Herrenfahrrad','type_highest_share'
    ]].sample(20)

ok great! We have calculated for each post code what type of bike was stolen the most.

## Retrieve the bike type with the highest share of thefts


In [ ]:
gdf_biketheft_res.shape

In [ ]:
# bike type with highest share of thefts
gdf_biketheft_res.groupby('type_highest_share')['thefts'].count() #.sort_values().tail(1)

In [ ]:
gdf_biketheft_res.groupby('type_highest_share').size() #.sort_values().tail(1)

In [ ]:
gdf_biketheft_res['type_highest_share'].value_counts()

It is __Herrenfahrrad with 438 thefts__ in the observed timeframe!  

---

In [63]:
my_columns = ['share_Herrenfahrrad','share_Damenfahrrad','share_Kinderfahrrad',
                                                     'share_Lastenfahrrad','share_Mountainbike','share_Rennrad',
                                                     'share_diverse Fahrräder','share_Fahrrad']

In [ ]:
fahrrad_max_share = gdf_biketheft_res[['share_Herrenfahrrad','share_Damenfahrrad','share_Kinderfahrrad',
                                                     'share_Lastenfahrrad','share_Mountainbike','share_Rennrad',
                                                     'share_diverse Fahrräder','share_Fahrrad']].max(axis=1)
fahrrad_max_share

In [ ]:
fahrrad_type = [ ",".join([col for col in my_columns if gdf_biketheft_res.loc[i,col]==mx]) for i,mx in fahrrad_max_share.items()]
fahrrad_type

In [ ]:
share_df = pd.DataFrame({"share":fahrrad_max_share, "kind":fahrrad_type}, index=gdf_biketheft_res.index)
share_df.tail(10)

In [ ]:
share_df['fahrrad_types'] = share_df['kind'].str.split(',')
share_df.tail(10)

In [ ]:
share_df_exploded = share_df.explode('fahrrad_types')
share_df_exploded.tail(10)

In [ ]:
# before we used the columns of gdf_bikethefts_res, and we also thefts and type_Herrenfahrrad ... as results below

share_df_exploded[share_df_exploded['fahrrad_types']=='thefts']

In [ ]:
share_df_exploded['fahrrad_types'].value_counts()

## Retrieve the postcode with the highest share of thefts


In [ ]:
gdf_biketheft_res['thefts'].max()

In [ ]:
# Retrieve the postcode with the highest share of thefts

gdf_biketheft_res[gdf_biketheft_res.thefts == gdf_biketheft_res.thefts.max()][['plr_name', 'avg_amount', 'thefts']]

It is __Alt-Treptow with 501 thefts__ in the observed timeframe with an average theft amount of 791 Euro!  

---

Congratulations!  
You made it through another intense notebook - but we hope the little excursions brought some fun ...